In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from collections import defaultdict
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import itertools
import pickle

In [3]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [4]:
def reduce_cols(X, components):
    total_variance = 0
    columns_variance = {}
    n_cols = X.shape[1]

    for col in X.columns:
        columns_variance[col] = X[col].var()
        total_variance += X[col].var()

    cols_to_drop = []
    for idx, (col, var) in enumerate(sorted(columns_variance.items(), key=lambda item: item[1])):
        if (n_cols - (idx + 1) >= components):
            cols_to_drop.append(col) 
        else:
            break

    return X.drop(columns=cols_to_drop)

new_X = reduce_cols(X, 250)
new_X.shape

(130, 250)

In [9]:
def nested_cv(X, components, mod_selection_score=accuracy_score):
    n_estimatorss = np.arange(50, 201, 35)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    best_overall_accuracy = 0
    best_overall_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo')

    X = reduce_cols(X, components)
    X = X.values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_score = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(n_estimatorss, contaminations, max_featuress):
                    clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    score = 0
                    if (mod_selection_score == precision_score):
                        score = mod_selection_score(true_values, pred_values, zero_division=0.0)
                    else:
                        score = mod_selection_score(true_values, pred_values)
                        
                    if score > best_score:
                        best_score = score
                        best_params['n_estimators'] = params[0]
                        best_params['contamination'] = params[1]
                        best_params['max_features'] = params[2]

            clf = IsolationForest(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            if accuracy_score(true_values, pred_values) > best_overall_accuracy:
                best_overall_accuracy = accuracy_score(true_values, pred_values)
                best_overall_params['n_estimators'] = best_params['n_estimators']
                best_overall_params['contamination'] = best_params['contamination']
                best_overall_params['max_features'] = best_params['max_features']


            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_estimators': best_overall_params['n_estimators'],
            'best contamination': best_overall_params['contamination'],
            'best max_features': best_overall_params['max_features'],
            'score used for model selection': mod_selection_score.__name__,
            'method used for model selection': 'nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'best overall accuracy': best_overall_accuracy * 100,
            'components': components}

In [15]:
results = nested_cv(X, 250, accuracy_score)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.55,accuracy_score,nested cv,66.153846,9.014562,79.74295,10.582023,51.904762,19.599158,84.615385,250


In [5]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [17]:
scores_df = add_record(scores_df, nested_cv(X, 250, f1_score))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.550,accuracy_score,nested cv,66.153846,9.014562,79.742950,10.582023,51.904762,19.599158,84.615385,250
1,LocalOutlierFactor,155,0.391111,0.775,f1_score,nested cv,66.666667,4.998356,75.592986,9.313196,59.523810,15.356681,76.923077,250


In [18]:
scores_df = add_record(scores_df, nested_cv(X, 250, precision_score))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.550,accuracy_score,nested cv,66.153846,9.014562,79.742950,10.582023,51.904762,19.599158,84.615385,250
1,LocalOutlierFactor,155,0.391111,0.775,f1_score,nested cv,66.666667,4.998356,75.592986,9.313196,59.523810,15.356681,76.923077,250
2,LocalOutlierFactor,155,0.500000,0.775,precision_score,nested cv,66.923077,10.204999,79.909091,12.338239,51.904762,16.176465,84.615385,250


In [19]:
scores_df = add_record(scores_df, nested_cv(X, 250, recall_score))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.550,accuracy_score,nested cv,66.153846,9.014562,79.742950,10.582023,51.904762,19.599158,84.615385,250
1,LocalOutlierFactor,155,0.391111,0.775,f1_score,nested cv,66.666667,4.998356,75.592986,9.313196,59.523810,15.356681,76.923077,250
2,LocalOutlierFactor,155,0.500000,0.775,precision_score,nested cv,66.923077,10.204999,79.909091,12.338239,51.904762,16.176465,84.615385,250
3,LocalOutlierFactor,50,0.010000,0.550,recall_score,nested cv,56.923077,2.878198,55.843691,1.803013,96.666667,4.416009,61.538462,250


In [22]:
scores_df = add_record(scores_df, nested_cv(X, 200, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 200, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 200, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 200, recall_score))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.550,accuracy_score,nested cv,66.153846,9.014562,79.742950,10.582023,51.904762,19.599158,84.615385,250
1,LocalOutlierFactor,155,0.391111,0.775,f1_score,nested cv,66.666667,4.998356,75.592986,9.313196,59.523810,15.356681,76.923077,250
2,LocalOutlierFactor,155,0.500000,0.775,precision_score,nested cv,66.923077,10.204999,79.909091,12.338239,51.904762,16.176465,84.615385,250
3,LocalOutlierFactor,50,0.010000,0.550,recall_score,nested cv,56.923077,2.878198,55.843691,1.803013,96.666667,4.416009,61.538462,250
4,LocalOutlierFactor,50,0.500000,0.775,accuracy_score,nested cv,66.410256,6.506963,79.318182,9.289704,51.904762,12.634761,76.923077,200
5,LocalOutlierFactor,190,0.282222,0.325,f1_score,nested cv,68.461538,6.466421,77.493243,9.831929,60.952381,13.768412,80.769231,200
6,LocalOutlierFactor,120,0.445556,0.775,precision_score,nested cv,66.153846,7.717906,80.438543,10.405859,50.952381,15.836466,88.461538,200
7,LocalOutlierFactor,50,0.010000,0.100,recall_score,nested cv,56.410256,4.134491,55.483612,2.281095,96.666667,5.128728,61.538462,200


In [23]:
scores_df = add_record(scores_df, nested_cv(X, 150, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 150, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 150, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 150, recall_score))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.550,accuracy_score,nested cv,66.153846,9.014562,79.742950,10.582023,51.904762,19.599158,84.615385,250
1,LocalOutlierFactor,155,0.391111,0.775,f1_score,nested cv,66.666667,4.998356,75.592986,9.313196,59.523810,15.356681,76.923077,250
2,LocalOutlierFactor,155,0.500000,0.775,precision_score,nested cv,66.923077,10.204999,79.909091,12.338239,51.904762,16.176465,84.615385,250
3,LocalOutlierFactor,50,0.010000,0.550,recall_score,nested cv,56.923077,2.878198,55.843691,1.803013,96.666667,4.416009,61.538462,250
4,LocalOutlierFactor,50,0.500000,0.775,accuracy_score,nested cv,66.410256,6.506963,79.318182,9.289704,51.904762,12.634761,76.923077,200
5,LocalOutlierFactor,190,0.282222,0.325,f1_score,nested cv,68.461538,6.466421,77.493243,9.831929,60.952381,13.768412,80.769231,200
6,LocalOutlierFactor,120,0.445556,0.775,precision_score,nested cv,66.153846,7.717906,80.438543,10.405859,50.952381,15.836466,88.461538,200
7,LocalOutlierFactor,50,0.010000,0.100,recall_score,nested cv,56.410256,4.134491,55.483612,2.281095,96.666667,5.128728,61.538462,200
8,LocalOutlierFactor,50,0.445556,0.325,accuracy_score,nested cv,65.897436,10.211440,77.212066,11.352889,51.428571,17.340135,88.461538,150
9,LocalOutlierFactor,85,0.282222,0.100,f1_score,nested cv,69.487179,6.946778,77.884753,9.461405,61.904762,12.688488,80.769231,150


In [27]:
scores_df = add_record(scores_df, nested_cv(X, 100, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 100, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 100, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 100, recall_score))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.550,accuracy_score,nested cv,66.153846,9.014562,79.742950,10.582023,51.904762,19.599158,84.615385,250
1,LocalOutlierFactor,155,0.391111,0.775,f1_score,nested cv,66.666667,4.998356,75.592986,9.313196,59.523810,15.356681,76.923077,250
2,LocalOutlierFactor,155,0.500000,0.775,precision_score,nested cv,66.923077,10.204999,79.909091,12.338239,51.904762,16.176465,84.615385,250
3,LocalOutlierFactor,50,0.010000,0.550,recall_score,nested cv,56.923077,2.878198,55.843691,1.803013,96.666667,4.416009,61.538462,250
4,LocalOutlierFactor,50,0.500000,0.775,accuracy_score,nested cv,66.410256,6.506963,79.318182,9.289704,51.904762,12.634761,76.923077,200
5,LocalOutlierFactor,190,0.282222,0.325,f1_score,nested cv,68.461538,6.466421,77.493243,9.831929,60.952381,13.768412,80.769231,200
6,LocalOutlierFactor,120,0.445556,0.775,precision_score,nested cv,66.153846,7.717906,80.438543,10.405859,50.952381,15.836466,88.461538,200
7,LocalOutlierFactor,50,0.010000,0.100,recall_score,nested cv,56.410256,4.134491,55.483612,2.281095,96.666667,5.128728,61.538462,200
8,LocalOutlierFactor,50,0.445556,0.325,accuracy_score,nested cv,65.897436,10.211440,77.212066,11.352889,51.428571,17.340135,88.461538,150
9,LocalOutlierFactor,85,0.282222,0.100,f1_score,nested cv,69.487179,6.946778,77.884753,9.461405,61.904762,12.688488,80.769231,150


In [10]:
scores_df = add_record(scores_df, nested_cv(X, 50, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 50, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 50, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 50, recall_score))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,LocalOutlierFactor,120,0.445556,0.550,accuracy_score,nested cv,66.153846,9.014562,79.742950,10.582023,51.904762,19.599158,84.615385,250
1,LocalOutlierFactor,155,0.391111,0.775,f1_score,nested cv,66.666667,4.998356,75.592986,9.313196,59.523810,15.356681,76.923077,250
2,LocalOutlierFactor,155,0.500000,0.775,precision_score,nested cv,66.923077,10.204999,79.909091,12.338239,51.904762,16.176465,84.615385,250
3,LocalOutlierFactor,50,0.010000,0.550,recall_score,nested cv,56.923077,2.878198,55.843691,1.803013,96.666667,4.416009,61.538462,250
4,LocalOutlierFactor,50,0.500000,0.775,accuracy_score,nested cv,66.410256,6.506963,79.318182,9.289704,51.904762,12.634761,76.923077,200
5,LocalOutlierFactor,190,0.282222,0.325,f1_score,nested cv,68.461538,6.466421,77.493243,9.831929,60.952381,13.768412,80.769231,200
6,LocalOutlierFactor,120,0.445556,0.775,precision_score,nested cv,66.153846,7.717906,80.438543,10.405859,50.952381,15.836466,88.461538,200
7,LocalOutlierFactor,50,0.010000,0.100,recall_score,nested cv,56.410256,4.134491,55.483612,2.281095,96.666667,5.128728,61.538462,200
8,LocalOutlierFactor,50,0.445556,0.325,accuracy_score,nested cv,65.897436,10.211440,77.212066,11.352889,51.428571,17.340135,88.461538,150
9,LocalOutlierFactor,85,0.282222,0.100,f1_score,nested cv,69.487179,6.946778,77.884753,9.461405,61.904762,12.688488,80.769231,150


In [11]:
file_path = 'if_exp2_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)